# Colab ile Yolo Modellerini Eğit

Drive hesabımızı colab'a bağlıyoruz.

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

My Drive içinde yolov4 diye klasör açıyoruz. Bu klasöre giriyoruz.

In [ ]:
cd /content/gdrive/My Drive/yolov4/

https://developer.nvidia.com/cudnn sitesinden cudnn 7.6.5 sürümünü indiriyoruz.(Cuda 10.1 için olanı)

In [ ]:
!tar -xzvf "cudnn-10.0-linux-x64-v7.6.5.32.tgz" -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

Yolo modellerini kullanabilmek için github üzerinden darknet'i indiriyoruz.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

GPU aktif ve hızlı kullanmak için gpu,cudnn,cudnn_half aktif ediyoruz. Modellerimizi video veya webcam üzerinden test etmek için, opencv'yi 0 yerine 1 yapıyoruz.

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

Değerleri 1 yaptıktan sonra make etmemiz gerekiyor. Aktif bir şekilde kullanmak için, make etmezsek yaptığımız değişiklik bir anlam ifade etmez.

In [ ]:
!make

Kullanmak istediğimiz yolo modellerini indiriyoruz. Test için .weights dosyasını train işlemleri için .conv dosyalarını indirmemiz gerekiyor.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

Colab üzerinden resimleri görebilmek için bir fonksiyon yazmamız gerekiyor.

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Veri setimizi train.txt ve test.txt olarak ayırmak için python scripti kullanıyoruz. Önce veri setimizin olduğu klasöre giriyoruz.

In [ ]:
cd dataset

Python scripti çalıştırıyoruz.

In [ ]:
!python train_test.py

Test veya eğitim yapabilmek için darknet klasörüne girmemiz gerekiyor. Eğer resim üzerinden test yapmak istiyorsak detector test yazıyoruz.

Video/webcam üzerinden test yapmak istiyorsak detector demo yazıyoruz.

In [ ]:
cd /content/gdrive/My Drive/yolov4/darknet/

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg
imShow('predictions.jpg')

Colab da direk video yansıtmak biraz zor işlem olacağından dolayı, video kaydetmesini istiyoruz.

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights video.mp4 -out_filename video_kaydet.avi -dont_show

Eğitim yapmak için cfg dosyasından düzenleme yapmanız gerekiyor. 

- subdivisions = 8 veya 16 yapın.
- width = 512 height = 512 değerini 32 katları olacak şekilde yapın.
- learning_rate = 0.001 # Eğitim için en stabil oran.
- max_batches = Nesne Tür Sayısı*2000
- step = max_batches %80, max_batches %90
- classes = Nesne Tür Sayısı # 3 yolo katmanındaki değerini değiştiriyoruz
- filters = (Nesne Tür Sayısı+5)*3 
- random=1 # Veri setini karıştırıp daha güzel sonuç almak için yapılır.


## .data ve .names Dosyalarını Düzenleme

.data dosyası için:
classes= 1
train  = /home/kadirnar/Desktop/darknet/data/daire/train.txt
valid  = /home/kadirnar/Desktop/darknet/data/daire/test.txt
names = /home/kadirnar/Desktop/darknet/cfg/daire.names
backup = /home/kadirnar/Desktop/darknet/backup

.names dosyası için:

Nesne ismi yazıp kaydedin.


Eğitim yapabilmek için gerekli dosya işlemleri yaptıktan sonra eğitim kodumuzu yazıp çalıştırıyoruz.

In [ ]:
!./darknet detector train uav.data cfg/uav.cfg yolov4-tiny.conv.29 -dont_show -map